In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [ ]:
%load_ext rpy2.ipython
%load_ext memory_profiler

R[write to console]: Error: cons memory exhausted (limit reached?)

R[write to console]: Error: no more error handlers available (recursive errors?); invoking 'abort' restart



In [ ]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../figshareausraincloud/raw_data/"

In [ ]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

In [ ]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [ ]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

In [ ]:
%%time
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
os.makedirs("../figshareausraincloud/processed_data/", exist_ok=True)
files = glob.glob('../figshareausraincloud/raw_data/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0).assign(model=re.findall(r'^.*?(?=_)', os.path.basename(file))[0])
                for file in files))
df.to_csv("../figshareausraincloud/processed_data/combined_data.csv")

In [2]:
df = pd.read_csv("../figshareausraincloud/processed_data/combined_data.csv")

In [3]:
df.head(10)

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2
5,1889-01-06 12:00:00,-36.25,-35.0,140.625,142.5,3.292260e-02,ACCESS-CM2
6,1889-01-07 12:00:00,-36.25,-35.0,140.625,142.5,8.906468e-02,ACCESS-CM2
7,1889-01-08 12:00:00,-36.25,-35.0,140.625,142.5,3.158012e-02,ACCESS-CM2
8,1889-01-09 12:00:00,-36.25,-35.0,140.625,142.5,3.107391e-02,ACCESS-CM2
9,1889-01-10 12:00:00,-36.25,-35.0,140.625,142.5,3.304103e-02,ACCESS-CM2


In [4]:
df.shape

(62513863, 7)